In [10]:
import os
import re
# os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"

In [14]:
from importlib import reload

import httpx
from openai import OpenAI

import src.agents as agents

In [6]:
client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

## ReAct prompt

In [34]:
react_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris
""".strip()

actions_re = re.compile('^Action: (\w+): (.*)')

## Actions

In [28]:
def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

def calculate(query: str):
    return eval(query)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [35]:
reload(agents)

def query_agent(query: str, max_turns: int = 10):
    i = 0
    bot = agents.BaseAgent(client=client, system_prompt=react_prompt)
    next_prompt = query
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [actions_re.match(a) for a in result.split('\n') if actions_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

print(query_agent("What does England share borders with?"))

Thought: England is part of the United Kingdom, and I should look up its geographical borders to learn what countries it shares borders with. Specifically, I will check if it shares borders with any countries or other regions. 

Action: wikipedia: England borders

PAUSE
 -- running wikipedia England borders
Observation: Anglo-Scottish <span class="searchmatch">border</span> (Scottish Gaelic: Crìochan Anglo-Albannach) is an internal <span class="searchmatch">border</span> of the United Kingdom separating Scotland and <span class="searchmatch">England</span> which runs for
Answer: England shares a border with Scotland.
Answer: England shares a border with Scotland.


In [36]:
print(query_agent("What is the age difference between the Queen and Prince Charles?"))

Thought: I need to find out the birth dates of both the Queen and Prince Charles to calculate the age difference between them. I should look up their birth dates on Wikipedia.  
Action: wikipedia: Queen Elizabeth II birth date  
PAUSE  
 -- running wikipedia Queen Elizabeth II birth date  
Observation: <span class="searchmatch">Elizabeth</span> <span class="searchmatch">II</span> (<span class="searchmatch">Elizabeth</span> Alexandra Mary; 21 April 1926 – 8 September 2022) was <span class="searchmatch">Queen</span> of the United Kingdom and other Commonwealth realms from 6 February
Thought: I have the birth date of Queen Elizabeth II, which is April 21, 1926. Now I need to find out the birth date of Prince Charles. I'll look it up on Wikipedia.  
Action: wikipedia: Prince Charles birth date  
PAUSE  
 -- running wikipedia Prince Charles birth date  
Observation: Isles, and <span class="searchmatch">Prince</span> and Great Steward of Scotland. The following year, <span class="searchmatch

In [38]:
from src.llm.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini", temperature=0)
print(llm.send_message({"role": "user", "content": "What is the capital of France?"}))

RetryError: RetryError[<Future at 0x10c64f280 state=finished raised BadRequestError>]